# Lab 3: Implementing Long-Running Tools with Human Approval

## Introduction

Welcome to Lab 3! In this lab, we'll enhance our asynchronous agent architecture to handle long-running operations - specifically, a human approval workflow. This pattern is essential for real-world AI applications where certain decisions require human oversight before proceeding.

By the end of this lab, you'll understand how to:
- Implement long-running tools in asynchronous agents
- Configure state persistence using DynamoDB
- Set up human-in-the-loop approval workflows
- Implement a complete asynchronous flow with state management
- Test the agent's ability to pause, wait for external input, and resume execution

## The Challenge of Long-Running Operations

Many real-world agent tasks require operations that:
- Take longer than function timeout limits
- Need human input or approval
- Must wait for external systems to complete processing
- Should maintain their state across multiple executions

Our solution to this challenge involves:
1. **Persisting agent state** in DynamoDB
2. **Notifying humans** via SNS when approval is needed
3. **Capturing responses** through an API Gateway
4. **Resuming execution** by placing results back in the agent's queue

## Architecture Components

Our enhanced architecture adds these components:

1. **DynamoDB Table (`AgentMemoryTable`)**:
   - Stores agent conversation state and context
   - Enables the agent to "remember" where it left off
   - Indexed by session_id and agent_name

2. **SNS Topic (`ApprovalNotificationTopic`)**:
   - Sends email notifications requesting human approval
   - Contains links to approve or deny the generated content

3. **API Gateway (`ApprovalAPI`)**:
   - Provides endpoints for humans to submit approvals/denials
   - Captures approval decisions and forwards them to a handler

4. **Approval Handler Lambda**:
   - Processes approval decisions from the API Gateway
   - Places the tool result back in the agent's SQS queue

Let's implement this architecture step by step to create a robust system capable of handling complex workflows that require human oversight.

⚠️⚠️ Please run the [prerequisites](../lab_0/prerequisites.ipynb) before continuing with this lab, if you haven't done so already. ⚠️⚠️

This lab assumes that:
- An ENVIRONMENT VARIABLE `PUBLISH_API_ENDPOINT` is set with the URL to use to publish to UniTok website.
- A valid `strands` AWS Lambda Layer exists in the AWS account.
- A valid AWS profile exists OR
- Valid AWS credentials are setup.

## Step 1: Setting Up Additional AWS Resources

First, we need to create the additional AWS resources required for our long-running tools architecture:

In [ ]:
# Import required libraries
import boto3

response = boto3.client('cloudformation').describe_stacks(StackName="UniTokStack")
        
# Get the Unito API endpoint URL
for output in response['Stacks'][0]['Outputs']:
    if output['OutputKey'] == "ApiEndpoint":
        PUBLISH_API_ENDPOINT = f'{output["OutputValue"]}posts'
    if output['OutputKey'] == "DistributionDomainName":
        UNITOK_URL = f'http://{output["OutputValue"]}'
    if output['OutputKey'] == "Strandslayer":
        STRANDS_LAYER_ARN = f'{output["OutputValue"]}'
print(f"Using following API endpoint for publishing UniTok posts: {PUBLISH_API_ENDPOINT}")
print(f"You can reach the UniTok site at : {UNITOK_URL}")
print(f"Using following Strands Lambda Layer: {STRANDS_LAYER_ARN}")


### Creating DynamoDB Table for Agent Memory
This memory table will store the messages of the agent along side it's session_id and any other attributes needed to resume its execution.
We will setup 2 keys:
 1. session_id (str) -> Primary key of the table
 2. agent_name (str) -> Secondary key of the table

Both will act as composite key to store and fetch records across agents

In [ ]:
# Create DynamoDB table for agent memory
dynamodb = boto3.client('dynamodb')

try:
    # Create the DynamoDB table for agent memory
    table_name = 'AgentMemoryTable'
    
    response = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'session_id', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'agent_name', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'session_id', 'AttributeType': 'S'},
            {'AttributeName': 'agent_name', 'AttributeType': 'S'}
        ],
        BillingMode='PAY_PER_REQUEST'
    )
    
    print(f"DynamoDB table created: {table_name}")
    
    # Wait for the table to be created
    print("Waiting for table to become active...")
    waiter = dynamodb.get_waiter('table_exists')
    waiter.wait(TableName=table_name)
    
except dynamodb.exceptions.ResourceInUseException:
    print(f"DynamoDB table {table_name} already exists")

### Creating SNS Topic for Approval Notifications
An SNS topic will be needed to send out emails requesting approval for the generated content from the human involved.
You will have to provide an email address which will be used to receive the approval notifications.

In [ ]:
# Create SNS topic for approval notifications
sns = boto3.client('sns')

try:
    # Create the SNS topic
    topic_name = 'ApprovalNotificationTopic'
    
    response = sns.create_topic(Name=topic_name)
    topic_arn = response['TopicArn']
    
    print(f"SNS topic created: {topic_arn}")
    
    # Ask for email to subscribe to the topic
    email = input("Enter an email address to receive approval notifications: ")
    
    # Subscribe the email to the topic
    subscription = sns.subscribe(
        TopicArn=topic_arn,
        Protocol='email',
        Endpoint=email
    )
    
    print(f"Subscription created: {subscription['SubscriptionArn']}")
    print(f"Please check your email {email} and confirm the subscription")
    
except Exception as e:
    print(f"Error creating SNS topic: {str(e)}")

### Creating API Gateway for Approval Responses
The email sent to the user will look something like this:

```markdown
Content Approval Request
The following content has been generated and requires your approval:      
`{content_to_approve}`
To approve: `{approve_url}`
To deny: `{deny_url}`
```

These approve and deny APIs are needed, for this we will now create API gateway and add resource methods to it.

Our api path will look like this

> Approve: `/approval/approve/{session_id}?toolUseId=<id of the tool used>`

> Deny: `/approval/deny/{session_id}?toolUseId=<id of the tool used>`

In [ ]:
# Create API Gateway for approval responses
apigw = boto3.client('apigateway')

try:
    # Create the API Gateway
    api_name = 'ApprovalAPI'
    
    # Create API
    api_response = apigw.create_rest_api(
        name=api_name,
        description='API for handling approval/denial responses',
        endpointConfiguration={
            'types': ['REGIONAL']
        }
    )
    
    api_id = api_response['id']
    print(f"API Gateway created: {api_id}")
    
    # Get the root resource ID
    resources = apigw.get_resources(restApiId=api_id)
    root_id = [resource for resource in resources['items'] if resource['path'] == '/'][0]['id']
    
    # Create a resource for approval path
    approval_resource = apigw.create_resource(
        restApiId=api_id,
        parentId=root_id,
        pathPart='approval'
    )
    approval_resource_id = approval_resource['id']
    
    # Create approve resource under approval
    approve_resource = apigw.create_resource(
        restApiId=api_id,
        parentId=approval_resource_id,
        pathPart='approve'
    )
    approve_resource_id = approve_resource['id']
    
    # Create session_id resource parameter under approve
    approve_session_resource = apigw.create_resource(
        restApiId=api_id,
        parentId=approve_resource_id,
        pathPart='{session_id}'
    )
    approve_session_resource_id = approve_session_resource['id']
    
    # Create deny resource under approval
    deny_resource = apigw.create_resource(
        restApiId=api_id,
        parentId=approval_resource_id,
        pathPart='deny'
    )
    deny_resource_id = deny_resource['id']
    
    # Create session_id resource parameter under deny
    deny_session_resource = apigw.create_resource(
        restApiId=api_id,
        parentId=deny_resource_id,
        pathPart='{session_id}'
    )
    deny_session_resource_id = deny_session_resource['id']
    
    # Create GET method for approve path with required query parameter
    apigw.put_method(
        restApiId=api_id,
        resourceId=approve_session_resource_id,
        httpMethod='GET',
        authorizationType='NONE',
        requestParameters={
            'method.request.querystring.toolUseId': True
        }
    )
    
    # Create GET method for deny path with required query parameter
    apigw.put_method(
        restApiId=api_id,
        resourceId=deny_session_resource_id,
        httpMethod='GET',
        authorizationType='NONE',
        requestParameters={
            'method.request.querystring.toolUseId': True
        }
    )
    
    print("API Gateway configured successfully, it will be connected to the Lambda function at a later step")
    
except Exception as e:
    print(f"Error creating API Gateway: {str(e)}")
    raise


## Step 2: Creating the Approval Handler Lambda

Now, let's create the Lambda function that will handle approval responses

In [ ]:
# Create directory for the approval handler Lambda
!mkdir -p ../functions/approval_handler/

Let's write the code for the `ApprovalHandler` Lambda function

In [ ]:
%%writefile ../functions/approval_handler/index.py
import json
import boto3
import os
from datetime import datetime
import logging

logger = logging.getLogger()

sqs = boto3.client('sqs')
queue_url = os.environ['SQS_QUEUE_URL']

def lambda_handler(event, context):
    try:
        # Parse the request path parameters
        session_id = event['pathParameters']['session_id']
        # Parse the request query parameters
        tool_use_id = event['queryStringParameters']['toolUseId']

        approval = 'approved' if 'approve' in event['resource'] else 'denied'
        is_approved = True if approval == 'approved' else False

        logger.info(f"Processing {approval} for session_id: {session_id}")
        
        # Send wake-up message to SQS
        message_body = {
            'session_id': session_id,
            'type': 'existing',
            'toolName': 'human_approval',
            'body': [{
                'toolResult': {
                    'toolUseId': tool_use_id,
                    'status': 'success',
                    'content': [{'text': approval}]
                }
            }]
        }

        sqs.send_message(
            QueueUrl=queue_url,
            MessageBody=json.dumps(message_body),
            MessageAttributes={
                'session_id': {
                    'StringValue': session_id,
                    'DataType': 'String'
                },
                'action': {
                    'StringValue': 'approval_response',
                    'DataType': 'String'
                },
                'tool_use_id': {
                    'StringValue': tool_use_id,
                    'DataType': 'String'
                }
            }
        )
        # Return a success response with HTML
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>{'Approval' if is_approved else 'Denial'} Confirmed</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 40px;
                    line-height: 1.6;
                }}
                .container {{
                    max-width: 600px;
                    margin: 0 auto;
                    padding: 20px;
                    border: 1px solid #ddd;
                    border-radius: 5px;
                }}
                h1 {{
                    color: {'#4CAF50' if is_approved else '#F44336'};
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Content {'Approved' if is_approved else 'Denied'}</h1>
                <p>You have successfully {'approved' if is_approved else 'denied'} the content.</p>
                <p>The agent will now {'proceed with publishing' if is_approved else 'discard the content'}.</p>
                <p>You can close this window.</p>
            </div>
        </body>
        </html>
        """

        # Return success page
        return {
            'statusCode': 200,
            'headers': {
                'Content-Type': 'text/html'
            },
            'body': html_content
        }

    except Exception as e:
        logger.error(f"Error processing approval: {str(e)}")
        return {
            'statusCode': 500,
            'headers': {'Content-Type': 'text/html'},
            'body': f"Failed with error {e}"
        }

The above code has to be packaged so that it can be deployed later.

In [ ]:
# Package the approval handler Lambda
def package_lambda_function(source_dir):
    """Package the Lambda function code into a zip file"""
    import zipfile
    import tempfile
    import os
    
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"Source directory {source_dir} not found")
    
    # Create a temporary directory for packaging
    with tempfile.TemporaryDirectory() as temp_dir:
        zip_path = os.path.join(temp_dir, 'function.zip')
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through all files in the source directory
            for root, _, files in os.walk(source_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    # Calculate the path within the zip file (relative to source_dir)
                    arcname = os.path.relpath(file_path, source_dir)
                    zipf.write(file_path, arcname)
        
        # Read the zip file content
        with open(zip_path, 'rb') as zip_file:
            zip_content = zip_file.read()
        
        return zip_content

The Lambda function will need permissions via an IAM role. Let's create a role with permissions to write to the SQS queue for the `post-generator-agent` created in Lab 2

In [ ]:

# Get the SQS queue URL
sqs_client = boto3.client('sqs')
post_generator_queue_url = sqs_client.get_queue_url(
    QueueName='post-generator-agent-tasks'
)['QueueUrl']

# Package the Lambda function
approval_handler_zip = package_lambda_function('../functions/approval_handler/')
print(f"Approval handler Lambda packaged: {len(approval_handler_zip) / 1024:.2f} KB")

# Create IAM role for the approval handler Lambda
iam = boto3.client('iam')

approval_handler_role_name = 'ApprovalHandlerRole'
approval_handler_policy = {
    "Version": "2012-10-17",
    "Statement": [
        # Basic Lambda execution
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        # SQS send message
        {
            "Effect": "Allow",
            "Action": [
                "sqs:SendMessage"
            ],
            "Resource": f"arn:aws:sqs:*:*:post-generator-agent-tasks"
        }
    ]
}

In [ ]:
# Create the IAM role
try:
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "lambda.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # Create or update the role
    try:
        role_response = iam.create_role(
            RoleName=approval_handler_role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        approval_handler_role_arn = role_response['Role']['Arn']
        print(f"Created IAM role: {approval_handler_role_arn}")
    except iam.exceptions.EntityAlreadyExistsException:
        role_response = iam.get_role(RoleName=approval_handler_role_name)
        approval_handler_role_arn = role_response['Role']['Arn']
        print(f"Using existing IAM role: {approval_handler_role_arn}")
    
    # Attach the policy
    iam.put_role_policy(
        RoleName=approval_handler_role_name,
        PolicyName=f"{approval_handler_role_name}-policy",
        PolicyDocument=json.dumps(approval_handler_policy)
    )
    
    # Wait for role to propagate
    print("Waiting for IAM role to propagate...")
    time.sleep(10)
    
except Exception as e:
    print(f"Error creating IAM role: {str(e)}")
    raise


Now that we have everything in place, let's deploy the function with the right permissions.

In [ ]:
# Deploy the approval handler Lambda
lambda_client = boto3.client('lambda')

approval_handler_name = 'ApprovalHandler'

try:
    # Check if the function already exists
    try:
        lambda_client.get_function(FunctionName=approval_handler_name)
        # Update the function
        lambda_client.update_function_code(
            FunctionName=approval_handler_name,
            ZipFile=approval_handler_zip
        )
        lambda_client.update_function_configuration(
            FunctionName=approval_handler_name,
            Runtime='python3.11',
            Role=approval_handler_role_arn,
            Handler='index.lambda_handler',
            Timeout=30,
            Environment={
                'Variables': {
                    'SQS_QUEUE_URL': post_generator_queue_url
                }
            }
        )
        print(f"Updated Lambda function: {approval_handler_name}")
    except lambda_client.exceptions.ResourceNotFoundException:
        # Create the function
        response = lambda_client.create_function(
            FunctionName=approval_handler_name,
            Runtime='python3.11',
            Role=approval_handler_role_arn,
            Handler='index.lambda_handler',
            Code={'ZipFile': approval_handler_zip},
            Timeout=30,
            Environment={
                'Variables': {
                    'SQS_QUEUE_URL': post_generator_queue_url
                }
            }
        )
        print(f"Created Lambda function: {approval_handler_name}")
    
    # Get the function ARN
    function_info = lambda_client.get_function(FunctionName=approval_handler_name)
    approval_handler_arn = function_info['Configuration']['FunctionArn']
    
except Exception as e:
    print(f"Error deploying Lambda function: {str(e)}")
    raise

## Step 3: Configuring API Gateway Integration

Now, let's connect our API Gateway to the approval handler Lambda:


In [ ]:
# Create Lambda integration for approve path
apigw.put_integration(
    restApiId=api_id,
    resourceId=approve_session_resource_id,
    httpMethod='GET',
    type='AWS_PROXY',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{boto3.session.Session().region_name}:lambda:path/2015-03-31/functions/{approval_handler_arn}/invocations'
)

# Create Lambda integration for deny path
apigw.put_integration(
    restApiId=api_id,
    resourceId=deny_session_resource_id,
    httpMethod='GET',
    type='AWS_PROXY',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{boto3.session.Session().region_name}:lambda:path/2015-03-31/functions/{approval_handler_arn}/invocations'
)

# Deploy the API to dev stage
deployment = apigw.create_deployment(
    restApiId=api_id,
    stageName='dev'
)

# Get the API endpoint
api_endpoint = f"https://{api_id}.execute-api.{boto3.session.Session().region_name}.amazonaws.com/dev"
print(f"API Gateway deployed: {api_endpoint}")

# Get the AWS account ID
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']

# Get the current region
region = boto3.session.Session().region_name

# Add Lambda permissions for API Gateway to invoke the function
lambda_client.add_permission(
    FunctionName='ApprovalHandler',
    StatementId=f'apigateway-approve-{int(time.time())}',
    Action='lambda:InvokeFunction',
    Principal='apigateway.amazonaws.com',
    SourceArn=f'arn:aws:execute-api:{region}:{account_id}:{api_id}/*/GET/approval/approve/*'
)

lambda_client.add_permission(
    FunctionName='ApprovalHandler',
    StatementId=f'apigateway-deny-{int(time.time())}',
    Action='lambda:InvokeFunction',
    Principal='apigateway.amazonaws.com',
    SourceArn=f'arn:aws:execute-api:{region}:{account_id}:{api_id}/*/GET/approval/deny/*'
)

print("API Gateway permissions configured successfully")

## Step 4: Updating the Post Generator Agent

Now, let's update our post generator agent to include the human approval tool:

In [ ]:
# Create directory for the updated post generator agent
!mkdir -p ../functions/post_generator_agent_v2/

The `post-generator-agent` Lambda function will have to be updated. Other than `index.py` a new file `human_approval.py` will be added, which is the new tool for the agent.

In [ ]:
%%writefile ../functions/post_generator_agent_v2/index.py
# Lambda function implementation of an async Strands Agent with human approval workflow
import json
import uuid
import boto3
import logging
import os
import requests
from strands import Agent, tool
from strands.models import BedrockModel
# Local imports
import human_approval

logger = logging.getLogger(__name__)

MEMORY_TABLE = os.environ.get('MEMORY_TABLE', None)
PUBLISH_API_ENDPOINT = os.environ.get('PUBLISH_API_ENDPOINT', None)
AGENT_NAME = 'post-generator-agent'

def save_to_agent_memory(session_id, messages):
    # Put messages () against the session_id in memory store
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(MEMORY_TABLE)
    logger.info(f"Saving {len(messages)} messages to agent memory for session_id {session_id}")
    agent_memory_object = {
        'session_id': session_id, 
        'agent_name': AGENT_NAME,
        'messages': messages,
    }
    
    table.put_item(Item=agent_memory_object)
    return True

def load_from_agent_memory(session_id):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(MEMORY_TABLE)
    logger.info(f"Loading messages from {AGENT_NAME} memory for session_id {session_id}")
    # Load messages from agent memory of given session_id for this AGENT_NAME
    response = table.get_item(Key={'session_id': session_id, 'agent_name': AGENT_NAME})
    item = response.get('Item', {})
    messages = item.get('messages', [])
    logger.info(f"Loaded {len(messages)} messages from agent memory of {AGENT_NAME} for session_id: {session_id}")
    return messages

def prepare(task) -> Agent:
    type = task.get('type', None)
    assert type is not None, "Task type is not specified"
    assert type in ["new", "existing"], "Task type is not supported, must be `new` or `existing`"
    logger.info(f"Preparing agent for {type} task")
    if type == "new":
        # Structure of a new task
        # {
        #     'type': 'new',
        #     'body': {
        #         'task': 'new task description',
        #     },
        # }
        task_body = task.get('body', None)
        assert task_body is not None, "Task body is not specified"
        task_description = task_body.get('task', None)
        assert task_description is not None, "Task description is not specified"
        # Create a new session_id UUID
        session_id = str(uuid.uuid4())
        logger.info(f"New session_id: {session_id}")
        # Create messages
        messages = []
        return session_id, messages, task_description
    
    if type == "existing":
        #  Result of successful tool execution
        # {
        #     'session_id': 'id of the session',
        #     'type': 'existing',
        #     'toolName': 'name of the tool',
        #     'body': [{
        #         'toolResult': {
        #             'toolUseId': 'id of the tool that was used',
        #             'status': 'success|error',
        #             'content': [{'text': 'tool result content | error message'}]
        #         }
        #     }]
        # }

        session_id = task.get('session_id', None)
        assert session_id is not None, "Session ID is not specified"
        logger.info(f"Using existing session_id: {session_id}")
        # Load messages from agent memory
        messages = load_from_agent_memory(session_id)
        if messages and len(messages) > 1:
            # Remove the last message from the messages
            messages = messages[:-1]
            # Append the tool result to the messages
            logger.info(f"Appending tool result to messages: {task.get('body', [{}])}")
            messages.append({
                "role": "user",
                "content": task.get('body', [{}])
            })
            return session_id, messages, "Continue"
        else:
            logger.info("No messages found in agent memory, starting a new conversation")
            return session_id, [], "Continue"
    logger.error(f"Unknown task type: {type}")
    return str(uuid.uuid4()), [], "Hello, how can you help?"

@tool
def publish_post(
    content: str, 
    author: str = "Unicorn Rentals", 
    unicorn_color: str = "rainbow", 
    image_url: str = None) -> str:
    """
    Publish a post to the UniTok social media platform.

    Args:
        content (str): The text content of the post.
        author (str, optional): The author of the post. Defaults to "Unicorn Rentals".
        unicorn_color (str, optional): The color of the unicorn. Choose from: pink, blue, purple, green, yellow, or rainbow. Defaults to "rainbow".
        image_url (str, optional): URL to an image to include with the post. Defaults to None.

    Returns:
        str: A message indicating the post was published successfully, or an error message.
    """

    # For this lab, we'll simulate posting to UniTok
    print(f"Publishing post to UniTok: {content}")
    print(f"Author: {author}")
    print(f"Unicorn Color: {unicorn_color}")
    if image_url:
        print(f"Image URL: {image_url}")
    post_data = {
        "content": content,
        "author": author,
        "unicornColor": unicorn_color
    }
    if image_url:
        post_data["imageUrl"] = image_url
    try:
        # Send the post to the API
        logger.info(f"Publishing post to UniTok: {post_data}")
        response = requests.post(PUBLISH_API_ENDPOINT, json=post_data)
        logger.info(f"Response from UniTok: {response}")
        # Check if the request was successful
        if response.status_code == 201:
            post_id = response.json().get("postId")
            logger.info(f"Post published successfully! Post ID: {post_id}")
            return f"Post published successfully! Post ID: {post_id}"
        else:
            logger.error(f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}")
            return f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}"
    except Exception as e:
        logger.error(f"Error publishing post with the exception {e}")
        return "Error publishing post with the exception {e}"


def lambda_handler(event, context):
    logger.info(f"Received event: {event}")
        
    # Even when processing a single message, AWS Lambda still wraps it in a Records array
    if not event.get('Records') or len(event['Records']) == 0:
        logger.error("No records found in the event")
        return {
            'statusCode': 400,
            'body': json.dumps('No SQS message records found in the event')
        }
    
    # Extract the first (and only) message
    record = json.loads(event['Records'][0]['body'])
    logger.info(f"Processing record: {record}")
    session_id, history, prompt = prepare(record)
    logger.info(f"Session ID: {session_id}, Prompt: {prompt}")
    system_prompt = """
    You are a creative social media manager for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

    Your task is to create engaging social media posts for UniTok, our unicorn-themed social media platform.
    Before publishing, you must request evaluation of the post to ensure your content adheres to our brand guidelines.

    Process for creating and publishing posts:
    1. Generate a creative post based on the user's request
    2. If the user prompted then request human approval of the post
    3. If there is no mention of human approval then directly publish it to our platform
    3. If the human approves the post then publish it to our platform
    4. If the human denies the post, then restart the process

    Important information about Unicorn Rentals:
    - We offer unicorns in various colors: pink, blue, purple, green, yellow, and rainbow (our most popular)
    - Our new product feature allows customers to pick their favorite color unicorn to rent
    - Our target audience includes families with children, fantasy enthusiasts, and event planners
    - Our brand voice is magical, playful, and family-friendly

    When creating posts:
    - Keep content family-friendly and positive
    - Highlight the magical experience of spending time with unicorns
    - Mention the new color selection feature when appropriate
    - Use emojis sparingly but effectively
    - Keep posts between 50-200 characters for optimal engagement
    - If your posts are continously being denied by humans then stop after 3 tries

    Always show your thought process when creating posts, evaluating them, and making revisions.
    """
    # Create model
    model = BedrockModel(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    )

    # Create agent
    agent = Agent(
        system_prompt=system_prompt,
        model=model,
        tools=[human_approval, publish_post],
        messages=history,
    )
    
    result = agent(prompt, session_id=session_id)

    if result.state.get("stop_event_loop", False):
        logger.info("Agent needs to wait for tool result. Saving state and sleeping.")
    save_to_agent_memory(session_id, agent.messages)
    logger.info(f"Agent message history {agent.messages}")
    logger.info(str(result))

In [ ]:
%%writefile ../functions/post_generator_agent_v2/human_approval.py
import logging
import os
import boto3
import json
from typing import Any
from botocore.exceptions import ClientError
from strands.types.tools import ToolResult, ToolUse

# Initialize logging and set paths
logger = logging.getLogger(__name__)
TOPIC_ARN = os.environ.get("TOPIC_ARN", None)
APPROVAL_API_ENDPOINT = os.environ.get("APPROVAL_API_ENDPOINT", None)

TOOL_SPEC = {
    "name": "human_approval",
    "description": "Request approval from a human for the content generated or critical decisions.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "content": {
                    "type": "string",
                    "description": "Content or decision that needs to be approved by the human."
                }
            },
            "required": ["content"]
        }
    }
}

def send_approval_email(content_to_approve, session_id, tool_use_id):
    """
    Sends an approval request email via SNS with approve/deny links
    
    Parameters:
    session_id (str): The id of the session currently running
    content_to_approve (str): The content that needs approval
    
    Returns:
    dict: Response from SNS publish or error information
    """
    try:
        assert session_id is not None, "Session ID is not specified"
        assert TOPIC_ARN is not None, "TOPIC_ARN is not specified"
        assert APPROVAL_API_ENDPOINT is not None, "APPROVAL_API_ENDPOINT is missing"
        # Create an SNS client
        sns_client = boto3.client('sns')
        
        # Create the approval and denial URLs
        approve_url = f"{APPROVAL_API_ENDPOINT}approve/{session_id}?toolUseId={tool_use_id}"
        deny_url = f"{APPROVAL_API_ENDPOINT}deny/{session_id}?toolUseId={tool_use_id}"
        
        # Plain text alternative for email clients that don't support HTML
        text_message = f"""
        Content Approval Request
        
        The following content has been generated and requires your approval:
        
        {content_to_approve}
        
        To approve: {approve_url}
        To deny: {deny_url}
        
        """
        
        # Create the message structure with both HTML and plain text versions
        message = {
            "default": text_message,
            "email": text_message,
            "email-json": json.dumps({
                "subject": "Content Approval Request",
                "body": {
                    "text": text_message
                }
            })
        }
        
        # Publish the message to the SNS topic
        response = sns_client.publish(
            TopicArn=TOPIC_ARN,
            Message=json.dumps(message),
            Subject="Content Approval Request",
            MessageStructure='json'
        )
        
        message = "An email has been sent successfully to request content approval"
        logging.info(message)
        return {
            "success": True,
            "message_id": response['MessageId'],
            "message": message,
        }
        
    except ClientError as e:
        logging.error(f"Error sending email: {e}")
        return {
            "success": False,
            "error": str(e)
        }


def human_approval(tool: ToolUse, **kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    content = tool["input"]["content"]
    request_state = kwargs.get("request_state", {})
    session_id = request_state.get('session_id', kwargs.get("session_id", None))

    logger.debug(f"Session ID: {session_id}")

    # Send out an SNS notification to request human feedback with content
    status = send_approval_email(content, session_id, tool_use_id)

    # Set the stop flag, so that the agent can sleep and store it's state in memory.
    request_state["stop_event_loop"] = True
    request_state["session_id"] = session_id

    return {
        "toolUseId": tool_use_id,
        "status": "success",
        "content": [{"text": status['message']}]
    }

Now let's package and deploy our updated post generator agent:

In [ ]:

approval_api_endpoint = f"https://{api_id}.execute-api.{boto3.session.Session().region_name}.amazonaws.com/dev/approval/"
# Package the Lambda function
post_generator_v2_zip = package_lambda_function('../functions/post_generator_agent_v2/')
print(f"Post Generator Agent V2 Lambda packaged: {len(post_generator_v2_zip) / 1024:.2f} KB")

# Update the IAM policy for the post generator agent
post_generator_role_name = 'PostGeneratorAgent-role'
post_generator_policy = {
    "Version": "2012-10-17",
    "Statement": [
        # Basic Lambda execution
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        # SQS poll from own queue
        {
            "Effect": "Allow",
            "Action": [
                "sqs:ReceiveMessage",
                "sqs:DeleteMessage",
                "sqs:GetQueueAttributes"
            ],
            "Resource": f"arn:aws:sqs:*:*:post-generator-agent-tasks"
        },
        # DynamoDB permissions
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem"
            ],
            "Resource": f"arn:aws:dynamodb:*:*:table/AgentMemoryTable"
        },
        # SNS permissions
        {
            "Effect": "Allow",
            "Action": [
                "sns:Publish"
            ],
            "Resource": topic_arn
        },
        # Bedrock permissions
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": "*"
        }
    ]
}

# Update the IAM role policy
try:
    iam = boto3.client('iam')
    iam.put_role_policy(
        RoleName=post_generator_role_name,
        PolicyName=f"{post_generator_role_name}-policy",
        PolicyDocument=json.dumps(post_generator_policy)
    )
    print(f"Updated IAM role policy for {post_generator_role_name}")
except Exception as e:
    print(f"Error updating IAM role policy: {str(e)}")
    raise


In [ ]:
import time
# Deploy the updated post generator agent
lambda_client = boto3.client('lambda')

try:
    # Update the function
    print("Updating the function code for PostGeneratorAgent")
    lambda_client.update_function_code(
        FunctionName='PostGeneratorAgent',
        ZipFile=post_generator_v2_zip
    )
    print("Waiting 10 seconds for the function to be updated")
    time.sleep(10)  # Wait for the function to be updated
    print("Updating the configuration to add new environment variables.")
    # Update the configuration
    lambda_client.update_function_configuration(
        FunctionName='PostGeneratorAgent',
        Environment={
            'Variables': {
                'PUBLISH_API_ENDPOINT': PUBLISH_API_ENDPOINT,
                'APPROVAL_API_ENDPOINT': approval_api_endpoint,
                'TOPIC_ARN': topic_arn,
                'MEMORY_TABLE': 'AgentMemoryTable'
            }
        }
    )
    
    print(f"Updated Lambda function: PostGeneratorAgent")
    
except Exception as e:
    print(f"Error updating Lambda function: {str(e)}")
    raise

## Step 5: Testing the Long-Running Tool Pattern

Now, let's test our implementation of the long-running tool pattern with human approval:

#### Scenario 1: No human approval requested

In [ ]:
sqs = boto3.client('sqs')

task_without_human_approval = "Create a promotional post for our new rainbow unicorn summer camp experience for kids ages 7-12."
task = {
  "type": "new",
  "body": {
    "task": task_without_human_approval
  }
}
sqs.send_message(
    QueueUrl=post_generator_queue_url,
    MessageBody=json.dumps(task)
)

#### Scenario 2: Post with human approval requested

In [ ]:
task_with_human_approval = "Create and request approval for an automn campagin for playful black unicorns for Halloween party"
task = {
  "type": "new",
  "body": {
    "task": task_with_human_approval
  }
}
sqs.send_message(
    QueueUrl=post_generator_queue_url,
    MessageBody=json.dumps(task)
)

In [ ]:
print(f"You can reach the UniTok site at : {UNITOK_URL}")

## Viewing Posts on the UniTok Website

Now that we've created and published posts using our agent, let's see them on the UniTok website!

To view your posts:
1. Use the **UniTokUrl** displayed above. It can also be found in your CloudFormation deployment output from prerequisites. It should look something like: `https://d123abc456def.cloudfront.net`
2. Open this URL in your web browser
3. You should see the posts that our agent has created and published, displayed in reverse chronological order

Each post shows:
- The content of the post
- The author (which we set to "Unicorn Rentals")
- The unicorn color (visualized with the appropriate color)
- The timestamp when the post was created
- The number of likes (starting at 0)

This demonstrates the end-to-end flow of our agent: it generates creative content based on our prompts, publishes it to the UniTok API, and then we can see the posts on the UniTok website.


## Conclusion

In this lab, we've successfully implemented a long-running tool pattern with human approval workflow for our asynchronous agent. This architecture allows our agent to:

1. Generate content based on user prompts
2. Request human approval before publishing
3. Persist its state while waiting for approval
4. Resume execution once approval is received
5. Complete the task by publishing the approved content

Key components we've implemented:

1. **DynamoDB Table**: For storing agent state between invocations
2. **SNS Topic**: For sending approval notifications via email
3. **API Gateway**: For capturing human approval decisions
4. **Approval Handler Lambda**: For processing approval responses
5. **Human Approval Tool**: For pausing agent execution and requesting approval

This pattern can be extended to many other long-running operations beyond human approval, such as:
- Waiting for batch processing jobs to complete
- Handling scheduled operations
- Coordinating multi-step workflows with external systems

By mastering this pattern, you can build agents that handle complex, real-world tasks that extend beyond the limitations of synchronous execution.